In [ ]:
# Define model parameters
input_dim = 6
hidden_dims = [128, 64]  # Two hidden layers with 128 and 64 units
output_dim = 40

# Initialize the model
model = MultiLayerNN(input_dim, hidden_dims, output_dim)

# Train the last layer weights using Moore-Penrose pseudo inverse
model.train_last_layer(X_train, Y_train)

# Create multiple models and average their predictions
num_models = 10
models = []

# Train multiple models
for _ in range(num_models):
    temp_model = MultiLayerNN(input_dim, hidden_dims, output_dim)
    temp_model.train_last_layer(X_train, Y_train)
    models.append(temp_model)

# Function to average predictions
def average_predictions(models, X):
    predictions = [model.predict(X) for model in models]
    return torch.mean(torch.stack(predictions), dim=0)

# Predict on the test set
avg_predictions = average_predictions(models, X_test)


# Select 10 random test samples
indices = np.random.choice(range(X_test.shape[0]), 10, replace=False)

plt.figure(figsize=(15, 10))
for i, idx in enumerate(indices):
    plt.subplot(5, 2, i + 1)
    plt.plot(Y_test[idx].numpy(), label="True Gains")
    plt.plot(avg_predictions[idx].detach().numpy(), label="Predicted Gains")
    plt.legend()
    plt.title(f"Sample {idx}")
plt.tight_layout()
plt.show()

# Compute prediction error
errors = Y_test - avg_predictions
errors = errors.detach().numpy().flatten()

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(errors, bins=50, alpha=0.7, color="blue", label="Prediction Errors")
plt.xlabel("Error")
plt.ylabel("Frequency")
plt.title("Histogram of Prediction Errors")
plt.legend()
plt.show()

# Compute Mean Squared Error on the test set
mse = torch.mean((avg_predictions - Y_test) ** 2)
print(f"Mean Squared Error on Test Set: {mse.item():.4f}")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


X = Gain_cell
Y = Raman_training

X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.4, random_state=42)
X_test.shape, X_train.shape , Y_test.shape, Y_train.shape

X_train = np.array(X_train)
X_test = np.array(X_test)
X_train_inv = torch.tensor(X_train, dtype=torch.float32)
X_test_inv = torch.tensor(X_test, dtype=torch.float32)

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_inv = torch.tensor(Y_train, dtype=torch.float32)
Y_test_inv = torch.tensor(Y_test, dtype=torch.float32)

In [ ]:
class MultiLayerNN_inverse:
    def __init__(self, input_dim, hidden_dims, output_dim):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim

        self.hidden_weights = [torch.randn(input_dim if i == 0 else hidden_dims[i - 1], hidden_dims[i]) for i in range(len(hidden_dims))]
        self.hidden_biases = [torch.randn(1, hidden_dims[i]) for i in range(len(hidden_dims))]
        self.output_weights = None

    def forward(self, X):
        activations = X
        for W, b in zip(self.hidden_weights, self.hidden_biases):
            activations = torch.relu(torch.matmul(activations, W) + b)
        return activations

    def train_last_layer(self, X, Y):
        hidden_output = self.forward(X)
        pseudo_inverse = torch.pinverse(hidden_output)
        self.output_weights = torch.matmul(pseudo_inverse, Y)

    def predict(self, X):
        hidden_output = self.forward(X)
        return torch.matmul(hidden_output, self.output_weights)

# Train multiple models for model averaging
num_models = 10
models = []

for _ in range(num_models):
    model = MultiLayerNN(input_dim=40, hidden_dims=[64, 64], output_dim=6)
    model.train_last_layer(X_train_inv, Y_train_inv)
    models.append(model)

# Model averaging function
def average_predictions(models, X):
    predictions = [model.predict(X) for model in models]
    return torch.mean(torch.stack(predictions), dim=0)

# Test predictions
avg_predictions = average_predictions(models, X_test_inv)  

predictions = model.predict(X_test_inv)
print("Test RMSE:", torch.sqrt(torch.mean((predictions - Y_test_inv) ** 2)))

In [ ]:
# Reverse the hstack and split the dataset column-wise
Raman_power_reversed = Raman_training[:, :3]
Raman_wavelength_reversed = Raman_training[:, 3:6]

# Convert to DataFrame for better visualization
df_power_reversed = pd.DataFrame(Raman_power_reversed, columns=['Power1', 'Power2', 'Power3'])
df_wavelength_reversed = pd.DataFrame(Raman_wavelength_reversed, columns=['Wavelength1', 'Wavelength2', 'Wavelength3'])

df_pump1 = Raman_power['Power 2']
print(df_pump1.head())

df_pump1_pred = Raman_power_reversed[:, 1]
df_pump1_pred = pd.DataFrame(df_pump1_pred, columns=['Predicted Power 1'])
print(df_pump1_pred.head())

plt.scatter(df_pump1, df_pump1_pred, alpha=0.5)